# Mem0 Agents

Mem0 (pronounced “mem-zero”) enhances AI assistants and agents with an intelligent memory layer, enabling personalized AI interactions. It remembers user preferences and traits and continuously updates over time, making it ideal for applications like customer support chatbots and AI assistants.


## 1. Setup


In [8]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import os
import warnings

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pydantic.*")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="mem0.*")
warnings.filterwarnings("ignore", message=".*deprecated.*")

MEM0_API_KEY = os.getenv("MEM0_API_KEY")

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")

In [3]:
from llama_index.memory.mem0 import Mem0Memory

context = {
    "user_id": "ASDRP",
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "mem0",
            "host": "localhost",
            "port": 6333,
            "embedding_model_dims": 1536,  # Change this according to your local model's dimensions
        },
    },
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4o",
            "temperature": 0.0,
            "max_tokens": 1500,
        },
    },
    "embedder": {
        "provider": "openai",
        "config": {"model": "text-embedding-3-small"},
    },
    "version": "v1.1",
}

mem0_memory = Mem0Memory.from_client(
    api_key=MEM0_API_KEY,
    context=context,
    search_msg_limit=4,
)

## 2. Mem0 Agents


In [5]:
def call_fn(name: str):
    """Call the provided name.
    Args:
        name: str (Name of the person)
    """
    print(f"Calling... {name}")


def email_fn(name: str):
    """Email the provided name.
    Args:
        name: str (Name of the person)
    """
    print(f"Emailing... {name}")

In [6]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    tools=[email_fn, call_fn],
    llm=llm,
)

In [9]:
response = await agent.run("My name is ASDRP.  What is your name?", memory=mem0_memory)
print(str(response))

/Users/pmui/SynologyDrive/research/2025/research2025/10-agent-memories/.venv/lib/python3.12/site-packages/mem0/client/main.py:34: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


I don't have a personal name like humans do, but you can call me Assistant. How can I assist you today, ASDRP?
